## Giới thiệu

Bài học này sẽ đề cập đến:
- Gọi hàm là gì và các trường hợp sử dụng của nó
- Cách tạo một lệnh gọi hàm bằng OpenAI
- Cách tích hợp lệnh gọi hàm vào một ứng dụng

## Mục tiêu học tập

Sau khi hoàn thành bài học này, bạn sẽ biết cách và hiểu được:

- Mục đích của việc sử dụng gọi hàm
- Thiết lập Gọi Hàm bằng Dịch vụ OpenAI
- Thiết kế các lệnh gọi hàm hiệu quả cho trường hợp sử dụng ứng dụng của bạn


## Hiểu về Gọi Hàm

Trong bài học này, chúng ta sẽ xây dựng một tính năng cho startup giáo dục cho phép người dùng sử dụng chatbot để tìm các khóa học kỹ thuật. Chúng ta sẽ đề xuất các khóa học phù hợp với trình độ kỹ năng, vai trò hiện tại và công nghệ mà họ quan tâm.

Để hoàn thành điều này, chúng ta sẽ kết hợp:
 - `OpenAI` để tạo trải nghiệm trò chuyện cho người dùng
 - `Microsoft Learn Catalog API` để giúp người dùng tìm kiếm các khóa học dựa trên yêu cầu của họ
 - `Function Calling` để lấy truy vấn của người dùng và gửi đến một hàm nhằm thực hiện yêu cầu API.

Để bắt đầu, hãy cùng tìm hiểu lý do tại sao chúng ta lại muốn sử dụng gọi hàm ngay từ đầu:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # lấy phản hồi mới từ GPT, nơi nó có thể xem phản hồi của hàm


print(second_response.choices[0].message)


### Tại sao cần Function Calling

Nếu bạn đã hoàn thành bất kỳ bài học nào khác trong khóa học này, có lẽ bạn đã hiểu sức mạnh của việc sử dụng các Mô hình Ngôn ngữ Lớn (LLMs). Hy vọng bạn cũng nhận ra một số hạn chế của chúng.

Function Calling là một tính năng của OpenAI Service được thiết kế để giải quyết các thách thức sau:

Định dạng phản hồi không nhất quán:
- Trước khi có function calling, các phản hồi từ mô hình ngôn ngữ lớn thường không có cấu trúc và thiếu nhất quán. Lập trình viên phải viết mã kiểm tra phức tạp để xử lý từng trường hợp khác nhau trong kết quả trả về.

Khó tích hợp với dữ liệu bên ngoài:
- Trước khi có tính năng này, việc đưa dữ liệu từ các phần khác của ứng dụng vào ngữ cảnh trò chuyện là rất khó khăn.

Bằng cách chuẩn hóa định dạng phản hồi và cho phép tích hợp mượt mà với dữ liệu bên ngoài, function calling giúp đơn giản hóa quá trình phát triển và giảm nhu cầu viết thêm các đoạn mã kiểm tra.

Người dùng không thể nhận được các câu trả lời như "Thời tiết hiện tại ở Stockholm như thế nào?". Nguyên nhân là do các mô hình chỉ giới hạn trong khoảng thời gian mà dữ liệu được huấn luyện.

Hãy cùng xem ví dụ dưới đây để minh họa cho vấn đề này:

Giả sử chúng ta muốn tạo một cơ sở dữ liệu về thông tin sinh viên để có thể gợi ý khóa học phù hợp cho họ. Dưới đây là hai mô tả về sinh viên có dữ liệu rất giống nhau.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Chúng ta muốn gửi dữ liệu này đến một LLM để phân tích. Sau đó, dữ liệu này có thể được sử dụng trong ứng dụng của chúng ta để gửi đến một API hoặc lưu trữ trong cơ sở dữ liệu.

Hãy tạo hai prompt giống hệt nhau để hướng dẫn LLM về những thông tin mà chúng ta quan tâm:


Chúng tôi muốn gửi điều này đến một LLM để phân tích các phần quan trọng đối với sản phẩm của chúng tôi. Vì vậy, chúng tôi có thể tạo hai lời nhắc giống hệt nhau để hướng dẫn LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Sau khi tạo hai lời nhắc này, chúng ta sẽ gửi chúng đến LLM bằng cách sử dụng `openai.ChatCompletion`. Chúng ta lưu lời nhắc vào biến `messages` và gán vai trò là `user`. Điều này nhằm mô phỏng một tin nhắn từ người dùng được gửi đến chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mặc dù các prompt giống nhau và mô tả cũng tương tự, chúng ta có thể nhận được các định dạng khác nhau của thuộc tính `Grades`.

Nếu bạn chạy ô trên nhiều lần, định dạng có thể là `3.7` hoặc `3.7 GPA`.

Điều này là do LLM nhận dữ liệu không có cấu trúc dưới dạng prompt được viết và cũng trả về dữ liệu không có cấu trúc. Chúng ta cần có một định dạng có cấu trúc để biết trước sẽ nhận được gì khi lưu trữ hoặc sử dụng dữ liệu này.

Bằng cách sử dụng gọi hàm (functional calling), chúng ta có thể đảm bảo nhận lại dữ liệu có cấu trúc. Khi sử dụng gọi hàm, LLM thực tế không gọi hoặc chạy bất kỳ hàm nào. Thay vào đó, chúng ta tạo ra một cấu trúc để LLM tuân theo khi trả lời. Sau đó, chúng ta sử dụng các phản hồi có cấu trúc đó để biết nên chạy hàm nào trong ứng dụng của mình.


![Sơ đồ luồng gọi hàm](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.vi.png)


### Các trường hợp sử dụng gọi hàm

**Gọi các công cụ bên ngoài**  
Chatbot rất hữu ích trong việc cung cấp câu trả lời cho các câu hỏi của người dùng. Bằng cách sử dụng gọi hàm, chatbot có thể dùng tin nhắn từ người dùng để thực hiện một số tác vụ nhất định. Ví dụ, một sinh viên có thể yêu cầu chatbot "Gửi email cho giảng viên của tôi nói rằng tôi cần thêm sự hỗ trợ với môn học này". Điều này có thể thực hiện bằng cách gọi hàm `send_email(to: string, body: string)`

**Tạo truy vấn API hoặc cơ sở dữ liệu**  
Người dùng có thể tìm kiếm thông tin bằng ngôn ngữ tự nhiên, sau đó được chuyển đổi thành một truy vấn hoặc yêu cầu API có định dạng. Ví dụ, một giáo viên có thể yêu cầu "Những học sinh nào đã hoàn thành bài tập cuối cùng" và điều này có thể gọi một hàm tên là `get_completed(student_name: string, assignment: int, current_status: string)`

**Tạo dữ liệu có cấu trúc**  
Người dùng có thể lấy một đoạn văn bản hoặc tệp CSV và sử dụng LLM để trích xuất thông tin quan trọng từ đó. Ví dụ, một sinh viên có thể chuyển một bài viết Wikipedia về các hiệp định hòa bình để tạo thẻ ghi nhớ AI. Việc này có thể thực hiện bằng cách sử dụng hàm `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Tạo Lệnh Gọi Hàm Đầu Tiên Của Bạn

Quy trình tạo một lệnh gọi hàm bao gồm 3 bước chính:
1. Gọi API Chat Completions với danh sách các hàm của bạn và một tin nhắn từ người dùng
2. Đọc phản hồi của mô hình để thực hiện một hành động, ví dụ như chạy một hàm hoặc gọi API
3. Gọi lại API Chat Completions với phản hồi từ hàm của bạn để sử dụng thông tin đó tạo phản hồi cho người dùng.


![Luồng của một Lời gọi Hàm](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.vi.png)


### Các thành phần của một lần gọi hàm

#### Đầu vào của người dùng

Bước đầu tiên là tạo một tin nhắn từ người dùng. Bạn có thể gán giá trị này một cách động bằng cách lấy giá trị từ một ô nhập liệu văn bản hoặc có thể gán giá trị trực tiếp tại đây. Nếu đây là lần đầu bạn làm việc với Chat Completions API, chúng ta cần xác định `role` và `content` của tin nhắn.

`role` có thể là `system` (tạo quy tắc), `assistant` (mô hình) hoặc `user` (người dùng cuối). Đối với việc gọi hàm, chúng ta sẽ gán là `user` cùng với một câu hỏi ví dụ.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Tạo hàm

Tiếp theo, chúng ta sẽ định nghĩa một hàm và các tham số của hàm đó. Ở đây, chúng ta sẽ chỉ sử dụng một hàm có tên là `search_courses`, nhưng bạn có thể tạo nhiều hàm khác nhau.

**Lưu ý quan trọng**: Các hàm sẽ được đưa vào thông điệp hệ thống gửi đến LLM và sẽ tính vào tổng số token mà bạn có thể sử dụng.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Định nghĩa**

Cấu trúc định nghĩa hàm có nhiều cấp độ, mỗi cấp có các thuộc tính riêng. Dưới đây là phần giải thích về cấu trúc lồng nhau này:

**Thuộc tính Hàm Cấp Cao Nhất:**

`name` - Tên của hàm mà bạn muốn được gọi.

`description` - Đây là phần mô tả cách hoạt động của hàm. Ở đây, điều quan trọng là phải cụ thể và rõ ràng.

`parameters` - Danh sách các giá trị và định dạng mà bạn muốn mô hình tạo ra trong phản hồi của nó.

**Thuộc tính Đối tượng Parameters:**

`type` - Kiểu dữ liệu của đối tượng parameters (thường là "object")

`properties` - Danh sách các giá trị cụ thể mà mô hình sẽ sử dụng cho phản hồi

**Thuộc tính của Từng Tham Số:**

`name` - Được xác định ngầm định bởi khóa thuộc tính (ví dụ: "role", "product", "level")

`type` - Kiểu dữ liệu của tham số cụ thể này (ví dụ: "string", "number", "boolean")

`description` - Mô tả về tham số cụ thể

**Thuộc tính Tùy chọn:**

`required` - Một mảng liệt kê những tham số nào là bắt buộc để hoàn thành việc gọi hàm


### Gọi hàm
Sau khi đã định nghĩa một hàm, bây giờ chúng ta cần đưa nó vào trong lệnh gọi tới Chat Completion API. Chúng ta làm điều này bằng cách thêm `functions` vào yêu cầu. Trong trường hợp này là `functions=functions`.

Ngoài ra còn có tùy chọn đặt `function_call` thành `auto`. Điều này có nghĩa là chúng ta sẽ để LLM tự quyết định nên gọi hàm nào dựa trên tin nhắn của người dùng thay vì tự mình chỉ định.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Bây giờ hãy cùng xem phản hồi và cách nó được định dạng:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Bạn có thể thấy tên của hàm được gọi và từ tin nhắn của người dùng, LLM đã có thể tìm dữ liệu để điền vào các tham số của hàm.


## 3. Tích hợp Gọi Hàm vào Ứng dụng.

Sau khi chúng ta đã kiểm tra phản hồi được định dạng từ LLM, bây giờ chúng ta có thể tích hợp nó vào một ứng dụng.

### Quản lý luồng xử lý

Để tích hợp điều này vào ứng dụng của mình, hãy thực hiện các bước sau:

Đầu tiên, hãy gọi tới dịch vụ OpenAI và lưu tin nhắn vào một biến có tên là `response_message`.


In [ ]:
response_message = response.choices[0].message

Bây giờ chúng ta sẽ định nghĩa hàm sẽ gọi API Microsoft Learn để lấy danh sách các khóa học:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Theo thông lệ tốt nhất, chúng ta sẽ kiểm tra xem mô hình có muốn gọi một hàm hay không. Sau đó, chúng ta sẽ tạo một trong các hàm có sẵn và ghép nó với hàm đang được gọi.
Tiếp theo, chúng ta sẽ lấy các đối số của hàm và ánh xạ chúng với các đối số từ LLM.

Cuối cùng, chúng ta sẽ thêm thông điệp gọi hàm và các giá trị được trả về bởi thông điệp `search_courses`. Điều này cung cấp cho LLM tất cả thông tin cần thiết để
phản hồi người dùng bằng ngôn ngữ tự nhiên.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Thử thách lập trình

Làm tốt lắm! Để tiếp tục học về OpenAI Function Calling, bạn có thể xây dựng: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Thêm nhiều tham số cho hàm để giúp người học tìm được nhiều khóa học hơn. Bạn có thể xem các tham số API có sẵn tại đây:
 - Tạo một hàm gọi khác nhận thêm thông tin từ người học như ngôn ngữ mẹ đẻ của họ
 - Xử lý lỗi khi hàm gọi hoặc API không trả về khóa học phù hợp



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
